In [6]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from TextPreprocessing import clean
import numpy as np
from nltk import sent_tokenize
import gensim
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
data=pd.read_csv("Squad1.csv")
df=pd.DataFrame(data[:75000])

In [8]:
def check(sentences):
    new_list=[]
    f=1
    while f==1:
        f=0
        for i in range(len(sentences)-1):
          #print(i)
            if len(sentences[i])==1:
                sentences[i-1]=sentences[i-1]+" " +sentences[i]
                sentences.pop(i)
                break;
            if (sentences[i].endswith('Op.') or sentences[i].endswith('No.') or sentences[i].endswith("!") or sentences[i].endswith(".)") or sentences[i].endswith("no.") or (65<=ord((sentences[i])[-2])<=90)) or sentences[i].endswith("Fr.") or sentences[i].endswith("Capt.")or sentences[i].endswith("Rev.")or sentences[i].endswith("est.")or sentences[i].endswith("app.")or sentences[i].endswith("Lt.")or sentences[i].endswith("Msgr.") or sentences[i].endswith("i.e.") or sentences[i].endswith("Ss.") or sentences[i].endswith("St.") or sentences[i].endswith("Sch.") or sentences[i].endswith("e.g.") or sentences[i].endswith("Mr.") or sentences[i].endswith("Mt.") or sentences[i].endswith("Ecl.") or sentences[i].endswith("Rs.") or sentences[i].endswith("9."):
                sentences[i]=sentences[i]+" " +sentences[i+1]
                sentences.pop(i+1)
                f=1
                break;
            if sentences[i].endswith("?"):
                sentences[i]=sentences[i]+sentences[i+1]
                sentences.pop(i+1)
                break;   
    return sentences
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["answer_text"] in x["sentences"][i]: idx = i
    return idx

In [175]:
df.tail()

,context,question,answer_text
74995,Puerto Rico has been discussed as a potential ...,What territory has been suggested as a new add...,Puerto Rico
74996,Puerto Rico has been discussed as a potential ...,What percentages of voters expressed unhappine...,54%
74997,Puerto Rico has been discussed as a potential ...,When did the legislature of Puerto Rico reques...,"December 11, 2012"
74998,"Since 1898, Puerto Rico has had limited repres...",How long has Puerto Rico had representation in...,Since 1898
74999,"Since 1898, Puerto Rico has had limited repres...",Who represents Puerto Rico in Congress?,a Resident Commissioner


In [176]:
import pickle
with open("doc2vec_sentences.pickle", "rb") as f:
    dict_emb_sent = pickle.load(f)
with open("doc2vec_questions.pickle", "rb") as f:
    dict_emb_ques = pickle.load(f)

In [177]:
df['sentences'] = df['context'].apply(lambda x: [i for i in check(sent_tokenize(x))])

In [178]:
c=0
df['sent_emb'] = df['sentences'].apply(lambda x: [dict_emb_sent[item] if item in dict_emb_sent else np.zeros(150) for item in x])
df['ques_emb'] = df['question'].apply(lambda x: [dict_emb_ques[x] if x in dict_emb_ques else np.zeros(150)] )

In [179]:
df.head(3)

,context,question,answer_text,sentences,sent_emb,ques_emb
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.79834604, -0.6132818, 0.09939866, -0.9515..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.7568376, 0.31501073, 0.15762527, 0.296937..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[-0.6069288, 0.2662351, 0.364287, -0.0608320..."


In [180]:
df["target"] = df.apply(get_target, axis = 1)
df.head()

,context,question,answer_text,sentences,sent_emb,ques_emb,target
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.79834604, -0.6132818, 0.09939866, -0.9515...",5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.7568376, 0.31501073, 0.15762527, 0.296937...",2
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[-0.6069288, 0.2662351, 0.364287, -0.0608320...",3
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.66225064, 0.65405315, -0.0076950206, 0.27...",4
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.857509, 0.23832044, 0.020176478, 0.366043...",1


In [181]:
def cosine(x):
    li=[]
    for i in x["sent_emb"]:
        s=i.reshape(1,-1)
        #print(s.shape)
        q=np.array(x["ques_emb"]).reshape(1,-1)
        #print(q.shape)
        li.append(float(cosine_similarity(s,q)))
    return li

In [182]:
%%time 
df["cosine_sim"]=df.apply(cosine,axis=1)

Wall time: 3min 40s


In [183]:
df.head(3)

,context,question,answer_text,sentences,sent_emb,ques_emb,target,cosine_sim
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.79834604, -0.6132818, 0.09939866, -0.9515...",5,"[-0.059910573065280914, 0.24365615844726562, 0..."
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.7568376, 0.31501073, 0.15762527, 0.296937...",2,"[0.005579862743616104, 0.4255257844924927, 0.3..."
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[-0.6069288, 0.2662351, 0.364287, -0.0608320...",3,"[0.06144462525844574, -0.005084047093987465, -..."


In [184]:
def cosine_index(d):
    return np.argmax(d)
def euc_index(d):
    return np.argmin(d)

In [185]:
#df["diff"] = (df["ques_emb"]- df["sent_emb"])**2
#df["euclidean_dis"] = df["diff"].apply(lambda x: list(np.sqrt(np.sum(x, axis = 1))))
#del df["diff"]

In [186]:
df["cosine_index"] = df["cosine_sim"].apply(lambda x: cosine_index(x))
#df["euc_index"] = df["euclidean_dis"].apply(lambda x: euc_index(x))

In [187]:
df.head()

,context,question,answer_text,sentences,sent_emb,ques_emb,target,cosine_sim,cosine_index
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.79834604, -0.6132818, 0.09939866, -0.9515...",5,"[-0.059910573065280914, 0.24365615844726562, 0...",5
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.7568376, 0.31501073, 0.15762527, 0.296937...",2,"[0.005579862743616104, 0.4255257844924927, 0.3...",3
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[-0.6069288, 0.2662351, 0.364287, -0.0608320...",3,"[0.06144462525844574, -0.005084047093987465, -...",3
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,a Marian place of prayer and reflection,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.66225064, 0.65405315, -0.0076950206, 0.27...",4,"[0.09212438762187958, -0.03000441938638687, -0...",4
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary,"[Architecturally, the school has a Catholic ch...","[[0.57688385, -0.28545356, -0.7204771, 0.77067...","[[[0.857509, 0.23832044, 0.020176478, 0.366043...",1,"[0.01529758982360363, 0.4713021516799927, 0.30...",3


In [188]:
def accuracy(target, predicted):
    acc = (target==predicted).sum()/len(target)
    return acc

In [189]:
print(accuracy(df["target"], df["cosine_index"]))
#print(accuracy(df["target"], df["euc_index"]))

0.4911066666666667


In [190]:
df.to_csv("Doc2Vec_with_embeddings.csv")